In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
from machine import freq
from common import task
import time
import random

In [3]:
frequencies = (
    40000000,
    80000000,
    160000000,
    240000000
)

def bench(cb, repeat, *args, **kwargs):
    for f in frequencies:
        freq(f)
        
        alg_time = []
        for i in range(repeat):
            time_start = time.ticks_us()
            cb(*args, **kwargs)
            time_end = time.ticks_us()
            
            alg_time.append(time.ticks_diff(time_end, time_start))
            
        yield {
            'freq': f,
            'min': min(alg_time),
            'max': max(alg_time),
            'avg': sum(alg_time)/len(alg_time)
        }    

In [4]:
"""
{
    80: {
        'int': (10, 20, 30),
        'float': (20, 50, 90)
    }
}
"""
def bench_sort_diff_types(elements, sort_cb):    
    cast = (int, float, bool)
    res = { freq:{} for freq in frequencies }

    
    for cast_cb in cast:
        test_set = (random.randint(0, 100000) for i in range(elements))
        casted_test_set = tuple(map(cast_cb, test_set))
        for m in tuple(bench(sort_cb, 1, casted_test_set)):
            res[m['freq']][cast_cb.__name__] = (m['min'], m['max'], m['avg'])
            
    return res



def print_bench_diff_types_result(res):
    type_cols = list(list(res.values())[0].keys())

    print('-' * (10 + 20 * len(type_cols) + len(type_cols)) )
    print(f'{"Freq":^10}| Time avg (us)')
    print(f'{"":10}', end='')
    for type_name in type_cols:
        print(f'|{type_name:^20}', end='')
    
    print()
    print('-' * (10 + 20 * len(type_cols) + len(type_cols)) )
        
    for freq, v in res.items():
        print(f'{freq/1000000:<10}', end='')
        for t in type_cols:
            print(f'|{int(res[freq][t][2]):20}', end='')
            
        print()
    print('-' * (10 + 20 * len(type_cols) + len(type_cols)) )


In [5]:
def bubbleSort(arr_in):
    arr = list(arr_in)
    n = len(arr)
 
    # Traverse through all array elements
    for i in range(n-1):
    # range(n) also work but outer loop will repeat one time more than needed.
 
        # Last i elements are already in place
        for j in range(0, n-i-1):
 
            # traverse the array from 0 to n-i-1
            # Swap if the element found is greater
            # than the next element
            if arr[j] > arr[j + 1] :
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr
        

        
@task('1: Bubble sort')
def task_1_bubble_sort():
    print(f'bubbleSort([5, 9, 8, 5, 6]) => {bubbleSort([5, 9, 8, 5, 6])}')
    
    element_counts = (50, 100, 200, 500)
    
    for cnt in element_counts:
        print(f'\nTest bubble sort for {cnt} elements:')
        res = bench_sort_diff_types(cnt, bubbleSort)
        print_bench_diff_types_result(res)
        

task_1_bubble_sort()    

**************************************************
Start Of Task #1: Bubble sort
**************************************************
bubbleSort([5, 9, 8, 5, 6]) => [5, 5, 6, 8, 9]

Test bubble sort for 50 elements:
-------------------------------------------------------------------------
   Freq   | Time avg (us)
          |       float        |        int         |        bool        
-------------------------------------------------------------------------
40        |               63146|               56824|               83217
80        |               31483|               28113|               41541
160       |               15845|               14162|               28104
240       |               10688|                9546|               23683
-------------------------------------------------------------------------

Test bubble sort for 100 elements:
-------------------------------------------------------------------------
   Freq   | Time avg (us)
          |       float        |

In [6]:
@task('2: Built-in sort')
def task_built_in_sort():
    element_counts = (50, 100, 1000, 2000)
    
    for cnt in element_counts:
        test_set = tuple( (random.randint(0, 100000) for i in range(cnt)) )
        
        print(f'\nTest built-in sort for {cnt} elements:')
        res = bench_sort_diff_types(cnt, sorted)
        print_bench_diff_types_result(res)

task_built_in_sort()

**************************************************
Start Of Task #2: Built-in sort
**************************************************

Test built-in sort for 50 elements:
-------------------------------------------------------------------------
   Freq   | Time avg (us)
          |       float        |        int         |        bool        
-------------------------------------------------------------------------
40        |                3441|                2137|                4311
80        |                1672|                 973|                2092
160       |                 911|                 555|                1135
240       |                 678|                 437|                 841
-------------------------------------------------------------------------

Test built-in sort for 100 elements:
-------------------------------------------------------------------------
   Freq   | Time avg (us)
          |       float        |        int         |        bool        